<a href="https://colab.research.google.com/github/RiddhimanRaut/Deep-Learning-based-CPR-estimation/blob/main/RNN_CPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

url = "https://github.com/RiddhimanRaut/Deep-Learning-based-CPR-estimation/blob/main/excel_dataset.xlsx?raw=true"
df = pd.read_excel(url)
df = df[5::6]
df.head()

date_time = df.

,Date Time,CPRAVG,PROP,BUT,ETHYLENE,AI12201M,FLW,COT,TEMP,PRESS,O2,DRAFT,ETH
5,01.01.2009 01:00:00,0.426381,19.491810,4.601716,30.532320,0.012364,30656.66992,819.507965,632.023209,0.467722,1.319569,-5.266271,74.667397
11,01.01.2009 02:00:00,0.424473,19.367620,4.547985,30.837540,0.011713,30590.67969,821.882797,632.649200,0.469376,1.111588,-4.903431,74.886887
17,01.01.2009 03:00:00,0.425650,19.237329,4.487629,30.659901,0.011062,30524.69922,822.465134,632.817825,0.466901,1.149024,-4.946926,75.145287
23,01.01.2009 04:00:00,0.427131,19.201321,4.403272,30.776039,0.011140,30527.44922,824.627869,634.766495,0.471751,1.014898,-4.618164,75.280724
29,01.01.2009 05:00:00,0.428690,17.998270,4.266049,31.351549,0.012562,30576.58008,825.376816,636.005157,0.470348,1.036991,-4.617743,76.774757
